In [39]:
from gurobipy import Model, GRB

In [40]:
# Create a new model
m = Model("bike_redistribution")

In [41]:
# Define the sets for hubs and edges including costs
hubs = range(1, 8)
edges_with_costs = {
    (1, 2): 20, (1, 5): 25,
    (2, 5): 30, (2, 7): 45,
    (3, 1): 20, (3, 6): 35,
    (4, 2): 30,
    (5, 3): 25, (5, 4): 15, (5, 6): 28,
    (6, 7): 12,
    (7, 4): 27
}

In [42]:
# Define supply/demand for each hub
hub_supply_demand = {1: -8, 2: 5, 3: 3, 4: -3, 5: 2, 6: -5, 7: 6}

In [43]:
# Create variables for each edge
x = {}
for i, j in edges_with_costs.keys():
    x[i, j] = m.addVar(vtype=GRB.INTEGER, name=f"x_{i}_{j}")

In [44]:
# Constraint: Supply-Demand balance for each hub
for hub in hubs:
    m.addConstr(
        sum(x[i, j] for i, j in edges_with_costs.keys() if i == hub) - 
        sum(x[j, i] for j, i in edges_with_costs.keys() if i == hub) == 
        hub_supply_demand[hub] * 14
    )

In [45]:
# Constraints for total number of bikes transferred from hubs 2 and 3
m.addConstr(sum(x[2, j] for j in hubs if (2, j) in edges_with_costs) <= 2 * sum(x[4, j] for j in hubs if (4, j) in edges_with_costs))
m.addConstr(sum(x[3, j] for j in hubs if (3, j) in edges_with_costs) <= 2 * sum(x[5, j] for j in hubs if (5, j) in edges_with_costs))

<gurobi.Constr *Awaiting Model Update*>

In [46]:
# Constraints for transfers between hubs 1-5
m.addConstr(
    sum(x[i, j] for i in range(1, 6) for j in range(1, 6) if (i, j) in edges_with_costs) >= 0.05 * 1400
)
m.addConstr(
    sum(x[i, j] for i in range(1, 6) for j in range(1, 6) if (i, j) in edges_with_costs) <= 0.50 * 1400
)

<gurobi.Constr *Awaiting Model Update*>

In [47]:
# Explicit Objective Function
m.setObjective(sum(edges_with_costs[i, j] * x[i, j] for i, j in edges_with_costs), GRB.MINIMIZE)

In [48]:
# Optimize the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11 rows, 12 columns and 46 nonzeros
Model fingerprint: 0xc9499dfa
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 7e+02]
Found heuristic solution: objective 25508.000000
Presolve removed 11 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 14938 25508 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.493800000000e+04, best bound 1.493800000000e+04, gap 0.0000%


In [49]:
# Output the solution
if m.status == GRB.Status.OPTIMAL:
    solution = m.getAttr('x', x)
    for i, j in edges_with_costs.keys():
        if solution[i, j] > 0:
            print(f"Transfer {solution[i, j]} bikes from hub {i} to hub {j}")

Transfer 140.0 bikes from hub 2 to hub 5
Transfer 112.0 bikes from hub 3 to hub 1
Transfer 70.0 bikes from hub 4 to hub 2
Transfer 70.0 bikes from hub 5 to hub 3
Transfer 28.0 bikes from hub 5 to hub 4
Transfer 70.0 bikes from hub 5 to hub 6
Transfer 84.0 bikes from hub 7 to hub 4


In [50]:
# Output the solution
if m.status == GRB.Status.OPTIMAL:
    solution = m.getAttr('x', x)
    for (i, j) in edges_with_costs:
        if solution[i, j] > 0:
            print(f"Transfer {solution[i, j]} bikes from hub {i} to hub {j}")
else:
    print('No optimal solution found')

Transfer 140.0 bikes from hub 2 to hub 5
Transfer 112.0 bikes from hub 3 to hub 1
Transfer 70.0 bikes from hub 4 to hub 2
Transfer 70.0 bikes from hub 5 to hub 3
Transfer 28.0 bikes from hub 5 to hub 4
Transfer 70.0 bikes from hub 5 to hub 6
Transfer 84.0 bikes from hub 7 to hub 4
